In [14]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

/disk2/fli/REVIVAL2


In [1]:
import pandas as pd
from scipy.stats import spearmanr

In [23]:
af3_joint = pd.read_csv("/disk2/fli/REVIVAL2/zs/bonddist/af3/struct_joint/PfTrpB-4bromo.csv")
af3_joint.head()

,AAs,fitness,var,n_mut,AA1,AA2,AA3,seq,enzyme,substrate,...,1:GLU-NH_1_avg,1:GLU-NH_1_std,2:GLU-NH_2_0,2:GLU-NH_2_1,2:GLU-NH_2_2,2:GLU-NH_2_3,2:GLU-NH_2_4,2:GLU-NH_2_agg,2:GLU-NH_2_avg,2:GLU-NH_2_std
0,AAV,-0.043328,I165A:I183A:Y301V,3,A,A,V,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,3.727298,0.790593,2.105471,2.095007,2.101589,4.163435,2.035116,2.095007,2.500123,0.832050
1,ADL,0.061846,I165A:I183D:Y301L,3,A,D,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,3.771857,0.923441,2.607558,2.055020,2.106428,4.068932,1.981890,2.055020,2.563966,0.784092
2,AGC,0.005751,I165A:I183G:Y301C,3,A,G,C,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,4.068228,0.803295,2.417483,2.407098,2.461830,4.527340,2.326060,2.407098,2.827962,0.850819
3,AGL,0.035926,I165A:I183G:Y301L,3,A,G,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,3.782819,0.858971,2.516566,2.038157,2.150245,4.153675,1.982800,2.038157,2.568288,0.814214
4,AMN,-0.002284,I165A:I183M:Y301N,3,A,M,N,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,3.526212,0.817171,1.872117,1.871327,1.858211,3.979154,1.809269,1.871327,2.278016,0.850879


In [25]:
af3_agg_columns = [c for c in af3_joint.columns if "agg" in c or "avg" in c]
af3_agg_columns

['0:C-C_agg',
 '0:C-C_avg',
 '1:GLU-NH_1_agg',
 '1:GLU-NH_1_avg',
 '2:GLU-NH_2_agg',
 '2:GLU-NH_2_avg']

In [26]:
for c in af3_agg_columns:
    print(c, spearmanr(af3_joint["fitness"], af3_joint[c]))

0:C-C_agg SignificanceResult(statistic=np.float64(-0.24586203090403558), pvalue=np.float64(0.0001151064942909508))
0:C-C_avg SignificanceResult(statistic=np.float64(-0.26867205843442693), pvalue=np.float64(2.3660663703723013e-05))
1:GLU-NH_1_agg SignificanceResult(statistic=np.float64(-0.3893119120058903), pvalue=np.float64(3.823141526737016e-10))
1:GLU-NH_1_avg SignificanceResult(statistic=np.float64(-0.3662288311325779), pvalue=np.float64(4.608611133003836e-09))
2:GLU-NH_2_agg SignificanceResult(statistic=np.float64(-0.41082568630395305), pvalue=np.float64(3.1359204118132095e-11))
2:GLU-NH_2_avg SignificanceResult(statistic=np.float64(-0.3827341984311275), pvalue=np.float64(7.927928658071465e-10))


In [29]:
chai_joint = pd.read_csv("/disk2/fli/REVIVAL2/zs/bonddist/chai/structure_joint/PfTrpB-4bromo.csv")
chai_joint.head()

,AAs,fitness,var,n_mut,AA1,AA2,AA3,seq,enzyme,substrate,...,1:GLU-NH_1_4,1:GLU-NH_1_avg,1:GLU-NH_1_std,2:GLU-NH_2_0,2:GLU-NH_2_1,2:GLU-NH_2_2,2:GLU-NH_2_3,2:GLU-NH_2_4,2:GLU-NH_2_avg,2:GLU-NH_2_std
0,AAV,-0.043328,I165A:I183A:Y301V,3,A,A,V,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,7.264118,6.776257,1.243588,2.281967,5.723475,5.446602,4.763861,5.118962,4.666973,1.234986
1,ADL,0.061846,I165A:I183D:Y301L,3,A,D,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,5.003357,5.313816,1.130466,5.452778,2.512928,2.526271,2.466567,2.862205,3.164150,1.152961
2,AGC,0.005751,I165A:I183G:Y301C,3,A,G,C,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,7.893424,7.140174,1.337229,2.351707,6.151737,5.680108,5.275805,5.772861,5.046444,1.375871
3,AGL,0.035926,I165A:I183G:Y301L,3,A,G,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,8.018167,7.332324,1.329792,2.680391,6.496720,6.366911,5.555043,6.455697,5.510952,1.456775
4,AMN,-0.002284,I165A:I183M:Y301N,3,A,M,N,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4bromo,...,6.941712,6.340353,1.020181,4.850679,3.750977,5.072831,2.308365,4.759011,4.148373,1.026088


In [30]:
chai_agg_columns = [c for c in chai_joint.columns if "agg" in c or "avg" in c]
chai_agg_columns

['0:C-C_avg', '1:GLU-NH_1_avg', '2:GLU-NH_2_avg']

In [31]:
for c in chai_agg_columns:
    print(c, spearmanr(chai_joint["fitness"], chai_joint[c]))

0:C-C_avg SignificanceResult(statistic=np.float64(-0.15619679987492238), pvalue=np.float64(0.015218579499025428))
1:GLU-NH_1_avg SignificanceResult(statistic=np.float64(-0.23068378001001294), pvalue=np.float64(0.000304564994911793))
2:GLU-NH_2_avg SignificanceResult(statistic=np.float64(-0.23219521813739086), pvalue=np.float64(0.0002772121462657627))


In [32]:
import pandas as pd
from glob import glob
from scipy.stats import spearmanr

from REVIVAL.util import get_file_name

for dock_type in ["joint", "seperate"]:

    # Initialize an empty list to store results
    results = []

    # Loop through the CSV files and calculate Spearman correlation
    for lib in sorted(glob(f"/disk2/fli/REVIVAL2/zs/bonddist/chai/structure_{dock_type}/*.csv")):
        lib_name = get_file_name(lib)
        lib_df = pd.read_csv(lib)
        
        df_nan = lib_df[lib_df.isna().any(axis=1)]
        if len(df_nan) > 0:
            print(f"NaN values found in {lib_name}")
            print(df_nan)
        avg_c = [c for c in lib_df.columns if "agg" in c or "avg" in c]
        # std_c = [c for c in chai_df.columns if "std" in c]

        # Calculate Spearman correlation for each column in avg_c
        for c in avg_c:
            correlation, p_value = spearmanr(lib_df["fitness"].values, lib_df[c].values)
            # std_col = merged_df[c.replace("avg", "std")].values
            results.append({
                "Library": lib_name,
                "Score_Type": c,
                # "variability": std_col.mean(),
                "Spearman_Correlation": correlation,
                "P_Value": p_value
            })

    # Convert results into a DataFrame and display it as a table
    results_df = pd.DataFrame(results)
    display(results_df)

,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_avg,-0.035191,0.437022
1,ParLQ,1:C-C_2_avg,0.010839,0.810859
2,PfTrpB-4bromo,0:C-C_avg,-0.156197,0.015219
3,PfTrpB-4bromo,1:GLU-NH_1_avg,-0.230684,0.000305
4,PfTrpB-4bromo,2:GLU-NH_2_avg,-0.232195,0.000277
5,PfTrpB-4cyano,0:C-C_avg,-0.061533,0.341515
6,PfTrpB-4cyano,1:GLU-NH_1_avg,0.236118,0.000217
7,PfTrpB-4cyano,2:GLU-NH_2_avg,0.230194,0.000314
8,PfTrpB-56chloro,0:C-C_avg,-0.210001,0.001039
9,PfTrpB-56chloro,1:GLU-NH_1_avg,0.024928,0.700207


,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_agg,0.169302,0.000166
1,ParLQ,1:C-C_2_agg,0.134358,0.002882
2,Rma-CB,0:C-B_agg,-0.333174,0.000031
3,Rma-CSi,0:C-Si_agg,-0.104207,0.204423


In [33]:
import pandas as pd
from glob import glob
from scipy.stats import spearmanr

from REVIVAL.util import get_file_name

for dock_type in ["joint", "seperate"]:

    # Initialize an empty list to store results
    results = []

    # Loop through the CSV files and calculate Spearman correlation
    for lib in sorted(glob(f"/disk2/fli/REVIVAL2/zs/bonddist/af3/struct_{dock_type}/*.csv")):
        lib_name = get_file_name(lib)
        lib_df = pd.read_csv(lib)
        
        df_nan = lib_df[lib_df.isna().any(axis=1)]
        if len(df_nan) > 0:
            print(f"NaN values found in {lib}")
            print(df_nan)
        avg_c = [c for c in lib_df.columns if "agg" in c or "avg" in c]
        # std_c = [c for c in chai_df.columns if "std" in c]

        # Calculate Spearman correlation for each column in avg_c
        for c in avg_c:
            lib_df_valid = lib_df.copy()
            valid_rows = lib_df_valid[["fitness", c]].dropna()
            correlation, p_value = spearmanr(valid_rows["fitness"].values, valid_rows[c].values)
            # correlation, p_value = spearmanr(lib_df["fitness"].values, lib_df[c].values)
            # std_col = merged_df[c.replace("avg", "std")].values
            results.append({
                "Library": lib_name,
                "Score_Type": c,
                # "variability": std_col.mean(),
                "Spearman_Correlation": correlation,
                "P_Value": p_value
            })

    # Convert results into a DataFrame and display it as a table
    results_df = pd.DataFrame(results)
    display(results_df)

NaN values found in /disk2/fli/REVIVAL2/zs/bonddist/af3/struct_joint/PfTrpB-56chloro.csv
    AAs   fitness                var  n_mut AA1 AA2 AA3  \
69  HML  0.071771  I165H:I183M:Y301L      3   H   M   L   

                                                  seq  enzyme substrate  ...  \
69  MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...  PfTrpB  56chloro  ...   

   1:GLU-NH_1_avg 1:GLU-NH_1_std 2:GLU-NH_2_0 2:GLU-NH_2_1 2:GLU-NH_2_2  \
69       5.389716       0.415514     3.958311     4.095618     4.223788   

   2:GLU-NH_2_3  2:GLU-NH_2_4  2:GLU-NH_2_agg  2:GLU-NH_2_avg  2:GLU-NH_2_std  
69     5.144954           NaN        4.095618        4.355668        0.465264  

[1 rows x 40 columns]
NaN values found in /disk2/fli/REVIVAL2/zs/bonddist/af3/struct_joint/PfTrpB-5iodo.csv
     AAs   fitness                var  n_mut AA1 AA2 AA3  \
144  RWC  0.016466  I165R:I183W:Y301C      3   R   W   C   

                                                   seq  enzyme substrate  ...  \
144  MWFG

,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_agg,-0.292502,4.023011e-11
1,ParLQ,0:C-C_1_avg,-0.176340,8.699198e-05
2,ParLQ,1:C-C_2_agg,-0.282574,1.898597e-10
3,ParLQ,1:C-C_2_avg,-0.185062,3.760488e-05
4,PfTrpB-4bromo,0:C-C_agg,-0.245862,1.151065e-04
...,...,...,...,...
69,PfTrpB-7methyl,2:GLU-NH_2_avg,-0.221975,5.176284e-04
70,Rma-CB,0:C-B_agg,0.083652,3.402773e-01
71,Rma-CB,0:C-B_avg,0.077319,3.782115e-01
72,Rma-CSi,0:C-Si_agg,0.070772,4.200196e-01


NaN values found in /disk2/fli/REVIVAL2/zs/bonddist/af3/struct_seperate/ParLQ.csv
       AAs    fitness  selectivity                       var  n_mut AA1 AA2  \
296  NMVDC   6.010976     0.243060  W56N:Y57M:L59V:Q60D:F89C      5   N   M   
297  NMVDV   6.128611     0.313192  W56N:Y57M:L59V:Q60D:F89V      5   N   M   
298  NNVEF  10.406167     0.400124       W56N:Y57N:L59V:Q60E      4   N   N   
299  NNVEQ   2.095577     0.061477  W56N:Y57N:L59V:Q60E:F89Q      5   N   N   
307  PCLTA   2.158520     0.084211       W56P:Y57C:Q60T:F89A      4   P   C   
308  PCLTD   0.917851     0.014363       W56P:Y57C:Q60T:F89D      4   P   C   
309  PCLTH   1.377055     0.144580       W56P:Y57C:Q60T:F89H      4   P   C   
326  PTGEL   2.126261     0.060992  W56P:Y57T:L59G:Q60E:F89L      5   P   T   
328  PTGER   4.675996     0.120611  W56P:Y57T:L59G:Q60E:F89R      5   P   T   
329  PTGES   2.256459     0.039850  W56P:Y57T:L59G:Q60E:F89S      5   P   T   
373  SAFRY  70.447102     4.014563  W56S:Y57A:L59

,Library,Score_Type,Spearman_Correlation,P_Value
0,ParLQ,0:C-C_1_agg,-0.169792,0.000195
1,ParLQ,0:C-C_1_avg,-0.137654,0.002588
2,ParLQ,1:C-C_2_agg,-0.141933,0.001887
3,ParLQ,1:C-C_2_avg,-0.088976,0.052133
4,Rma-CB,0:C-B_agg,-0.009152,0.916417
5,Rma-CB,0:C-B_avg,-0.006001,0.945132
6,Rma-CSi,0:C-Si_agg,0.069546,0.422841
7,Rma-CSi,0:C-Si_avg,0.186394,0.030420
